## Function Calling with Ollama

### Requirements

#### 1. Ollama

Ollama installation instructions per OS (macOS, Linux, Windows) can be found on [their website](https://ollama.com/download). For Linux simply (run cell below if not installed): 

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

#### 2. Python Ollama Library

For that:

In [ ]:
%pip install ollama

#### 3. Pull the model from Ollama

Download the q8 quantized NousHermes-2-Pro-Mistral-7B from Ollama (uploaded by adrienbrault):

In [ ]:
!ollama pull interstellarninja/hermes-2-theta-llama-3-8b:latest

### Usage

#### 1. Define Tools

In [199]:
import requests
import random
from datetime import datetime
import pytz

def get_weather_forecast(location: str) -> dict[str, str]:
    """Retrieves a simple weather forecast for a given location"""
    url = f"https://wttr.in/{location}?format=%C,%t"
    response = requests.get(url)
    if response.status_code == 200:
        condition, temperature = response.text.strip().split(',')
        return {
            "location": location,
            "forecast": condition,
            "temperature": temperature
        }
    else:
        return {"error": "Unable to fetch weather data"}


def get_stock_price(symbol: str) -> float:
    """Retrieves the stock price for a given symbol"""
    api_key = "your_stock_api_key"
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    return float(data["Global Quote"]["05. price"])

def get_random_number(min_value: int, max_value: int) -> int:
    """Returns a random number between min_value and max_value"""
    return random.randint(min_value, max_value)

def get_current_time(time_zone: str, format: str) -> str:
    """Returns the current time in the specified time zone and format"""
    tz = pytz.timezone(time_zone)
    current_time = datetime.now(tz)
    return current_time.strftime(format)

def get_user_location(accuracy: int) -> str:
    """Returns the user's location based on the provided IP address and accuracy level"""
    ip_address = socket.gethostbyname(socket.gethostname())
    url = f"http://ip-api.com/json/{ip_address}"
    response = requests.get(url)
    data = response.json()
    if accuracy == 1:
        return data["country"]
    elif accuracy == 2:
        return f"{data['city']}, {data['country']}"
    else:
        return f"{data['city']}, {data['region']}, {data['country']}"

In [200]:
def test_functions():
    print("Testing get_weather_forecast:")
    try:
        weather = get_weather_forecast("London")
        print(f"Weather in London: {weather}")
    except Exception as e:
        print(f"Error in get_weather_forecast: {str(e)}")

    print("\nTesting get_stock_price:")
    try:
        price = get_stock_price("AAPL")
        print(f"Current price of AAPL: ${price:.2f}")
    except Exception as e:
        print(f"Error in get_stock_price: {str(e)}")

    print("\nTesting get_random_number:")
    try:
        number = get_random_number(2, 42)
        print(f"Random number between 1 and 100: {number}")
    except Exception as e:
        print(f"Error in get_random_number: {str(e)}")

    print("\nTesting get_current_time:")
    try:
        time = get_current_time("America/New_York", "%Y-%m-%d %H:%M:%S")
        print(f"Current time in New York: {time}")
    except Exception as e:
        print(f"Error in get_current_time: {str(e)}")

    print("\nTesting get_user_location:")
    try:
        location = get_user_location("8.8.8.8", 2)
        print(f"Location for IP 8.8.8.8: {location}")
    except Exception as e:
        print(f"Error in get_user_location: {str(e)}")

In [201]:
test_functions()

Testing get_weather_forecast:
Weather in London: {'location': 'London', 'forecast': 'Partly cloudy ', 'temperature': '+22°C'}

Testing get_stock_price:
Current price of AAPL: $224.31

Testing get_random_number:
Random number between 1 and 100: 10

Testing get_current_time:
Current time in New York: 2024-07-21 09:49:44

Testing get_user_location:
Location for IP 8.8.8.8: Ashburn, United States


In [202]:
from langchain_core.utils.function_calling import convert_to_openai_tool

functions = [
    get_weather_forecast,
    get_stock_price,
    get_random_number,
    get_current_time,
    get_user_location
]

tools = [convert_to_openai_tool(t) for t in functions]

In [203]:
import json


print(json.dumps(tools, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "get_weather_forecast",
      "description": "Retrieves a simple weather forecast for a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string"
          }
        },
        "required": [
          "location"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_stock_price",
      "description": "Retrieves the stock price for a given symbol",
      "parameters": {
        "type": "object",
        "properties": {
          "symbol": {
            "type": "string"
          }
        },
        "required": [
          "symbol"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_random_number",
      "description": "Returns a random number between min_value and max_value",
      "parameters": {
        "type": "object",
        "properties": {
          "min_v

In [204]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [207]:
messages = [
    {"role": "user", "content": "Get the user location and the weather forecast for user's location"}
]

In [189]:
messages = [
    {"role": "user", "content": "get me a random number between 1 and 100"}
]

In [149]:
messages = [
    {"role": "user", "content": "what's the weather forecast for paris for 5 days"}
]

In [208]:
response = client.chat.completions.create(
    model="hermes-2-pro-llama-3-8b-tools",
    messages = messages,
    tools=tools
)

In [209]:
print(response)

ChatCompletion(id='chatcmpl-542', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To retrieve both user location and local weather forecast, I'll need an IP address to determine your location. Could you please provide me with your IP address? Also, what accuracy level would you like for your location: 1 (city), 2 (street), or 3 (precise) ?", role='assistant', function_call=None, tool_calls=None))], created=1721569879, model='hermes-2-pro-llama-3-8b-tools', object='chat.completion', system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=62, prompt_tokens=386, total_tokens=448))


In [190]:
assistant_message = response.choices[0].message
messages.append(assistant_message)
print(messages)

[{'role': 'user', 'content': 'get me a random number between 1 and 100'}, ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bh3z51i2', function=Function(arguments='{"max_value":100,"min_value":1}', name='quantum_rng'), type='function'), ChatCompletionMessageToolCall(id='call_k8ldjn2s', function=Function(arguments='null', name=''), type='function')])]


In [191]:
if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        function_call = tool_call.function
        name = function_call.name
        arguments = json.loads(function_call.arguments)
        for function in functions:
            if function.__name__ == name:
                result = function(**arguments)  # Call the function and collect the results
                print(result)
                result_content = {
                    "name": name,
                    "content": result
                }
                messages.append(
                    {
                        "role": "tool",
                        "content": json.dumps(result_content),
                        "tool_call_id": tool_call.id
                    }
                )
                break

14


In [192]:
messages

[{'role': 'user', 'content': 'get me a random number between 1 and 100'},
 ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bh3z51i2', function=Function(arguments='{"max_value":100,"min_value":1}', name='quantum_rng'), type='function'), ChatCompletionMessageToolCall(id='call_k8ldjn2s', function=Function(arguments='null', name=''), type='function')]),
 {'role': 'tool',
  'content': '{"name": "quantum_rng", "content": 14}',
  'tool_call_id': 'call_bh3z51i2'}]

In [194]:
response = client.chat.completions.create(
    model="hermes-2-pro-llama-3-8b-tools",
    messages = messages,
    tools=tools)

In [197]:
print(response.choices[0].message)

ChatCompletionMessage(content='Here is the random number between 1 and 100: 84. Would you like to do anything else?', role='assistant', function_call=None, tool_calls=None)
